In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install PyMuPDF
!pip install sentence_transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 68.1 MB/s eta 0:00:00


In [ ]:
import os
import json
import fitz
import re
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("sentence-transformers/bert-base-nli-mean-tokens")

dossier_articles = "/content/drive/MyDrive/ressources"
nom_fichier_index = "/content/drive/MyDrive/index_bert_pages.json"
index_vecteurs = {}

def nettoyer_texte(texte):
    texte = texte.lower()
    texte = re.sub(r'[^a-zA-Z0-9\s]', ' ', texte)
    texte = re.sub(r'\s+', ' ', texte)
    return texte.strip()

if os.path.exists(nom_fichier_index):
    with open(nom_fichier_index, "r", encoding="utf-8") as f:
        index_vecteurs = json.load(f)
else:
    fichiers = [f for f in os.listdir(dossier_articles) if f.endswith(".pdf")]

    for fichier in fichiers:
        chemin = os.path.join(dossier_articles, fichier)

        try:
            pdf = fitz.open(chemin)
        except Exception as e:
            print(f"Error reading {fichier}: {e}")
            continue

        print(f"Indexing {fichier} ({pdf.page_count} pages)...")

        for page_num, page in enumerate(pdf, start=1):
            texte_page = page.get_text()
            texte_nettoye = nettoyer_texte(texte_page)

            if texte_nettoye.strip() == "":
                continue
            # Embedding MiniLM
            vecteur = model.encode(texte_nettoye).tolist()

            # Stocker avec clé "fichier_page"
            key = f"{fichier}_page_{page_num}"
            index_vecteurs[key] = vecteur

            print(f"Indexed: {key}")

        pdf.close()

    # Sauvegarder index
    with open(nom_fichier_index, "w", encoding="utf-8") as f:
        json.dump(index_vecteurs, f, indent=2, ensure_ascii=False)

print(f"MiniLM index ready! {len(index_vecteurs)} page vectors embedded.")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Indexing 1 The Comedy of Errors author William Shakespeare.pdf (83 pages)...
Indexed: 1 The Comedy of Errors author William Shakespeare.pdf_page_2
Indexed: 1 The Comedy of Errors author William Shakespeare.pdf_page_3
Indexed: 1 The Comedy of Errors author William Shakespeare.pdf_page_4
Indexed: 1 The Comedy of Errors author William Shakespeare.pdf_page_5
Indexed: 1 The Comedy of Errors author William Shakespeare.pdf_page_6
Indexed: 1 The Comedy of Errors author William Shakespeare.pdf_page_7
Indexed: 1 The Comedy of Errors author William Shakespeare.pdf_page_8
Indexed: 1 The Comedy of Errors author William Shakespeare.pdf_page_9
Indexed: 1 The Comedy of Errors author William Shakespeare.pdf_page_10
Indexed: 1 The Comedy of Errors author William Shakespeare.pdf_page_11
Indexed: 1 The Comedy of Errors author William Shakespeare.pdf_page_12
Indexed: 1 The Comedy of Errors author William Shakespeare.pdf_page_13
Indexed: 1 The Comedy of Errors author William Shakespeare.pdf_page_14
Indexed:

In [ ]:
import os
import json
import fitz
import re
from sentence_transformers import SentenceTransformer

# === Charger modèle ===
model = SentenceTransformer("sentence-transformers/bert-base-nli-mean-tokens")

# === Dossiers ===
dossier_articles = "/content/drive/MyDrive/ressources"
nom_fichier_index_livres = "/content/drive/MyDrive/index_bert_books.json"

# === Fonctions utilitaires ===
def nettoyer_texte(texte):
    texte = texte.lower()
    texte = re.sub(r'[^a-zA-Z0-9\s]', ' ', texte)
    texte = re.sub(r'\s+', ' ', texte)
    return texte.strip()

# === Index final (un vecteur par livre) ===
index_livres = {}

# ====== INDEXATION ======
fichiers = [f for f in os.listdir(dossier_articles) if f.endswith(".pdf")]

for fichier in fichiers:
    chemin = os.path.join(dossier_articles, fichier)

    print(f"\n📘 Traitement du livre : {fichier}")

    try:
        pdf = fitz.open(chemin)
    except Exception as e:
        print(f"Impossible d’ouvrir {fichier}: {e}")
        continue

    # Concaténer toutes les pages du PDF
    texte_total = ""
    for page in pdf:
        texte_page = page.get_text()
        texte_total += " " + texte_page

    pdf.close()

    # Nettoyage du texte
    texte_nettoye = nettoyer_texte(texte_total)

    if texte_nettoye.strip() == "":
        print(f"⚠️ Livre vide : {fichier}")
        continue

    # Embedding du livre entier
    print("   → Génération embedding...")
    vecteur = model.encode(texte_nettoye).tolist()

    # Enregistrer dans l’index
    index_livres[fichier] = vecteur

print("\n💾 Sauvegarde des embeddings...")
with open(nom_fichier_index_livres, "w", encoding="utf-8") as f:
    json.dump(index_livres, f, indent=2, ensure_ascii=False)

print(f"\n✅ Indexation terminée : {len(index_livres)} livres indexés !")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]


📘 Traitement du livre : 1 The Comedy of Errors author William Shakespeare.pdf
   → Génération embedding...

📘 Traitement du livre : 2 Titus Andronicus author William Shakespeare.pdf
   → Génération embedding...

📘 Traitement du livre : 3 The Rape of Lucrece author William Shakespeare.pdf
   → Génération embedding...

📘 Traitement du livre : 4 The Winter's Tale author William Shakespeare.pdf
   → Génération embedding...

📘 Traitement du livre : 5 King John author William Shakespeare.pdf
   → Génération embedding...

📘 Traitement du livre : 6 A Midsummer Night's Dream author William Shakespeare.pdf
   → Génération embedding...

📘 Traitement du livre : 8 The Tragedy of Julius Caesar author William Shakespeare.pdf
   → Génération embedding...

📘 Traitement du livre : 9 As You Like It author William Shakespeare.pdf
   → Génération embedding...

📘 Traitement du livre : 10 The Tempest author William Shakespeare.pdf
   → Génération embedding...

📘 Traitement du livre : 11 The Tragedy of Hamle

In [ ]:
import json
import torch
from sentence_transformers import SentenceTransformer, util

# ===== Charger MiniLM =====
model = SentenceTransformer("sentence-transformers/bert-base-nli-mean-tokens")

# ===== Charger l'index =====
with open("/content/drive/MyDrive/index_minilm_pages.json", "r", encoding="utf-8") as f:
    index_vecteurs_raw = json.load(f)

# Convertir tous les vecteurs JSON en torch.float32 pour accélérer la recherche
index_vecteurs = {}
for doc, vec in index_vecteurs_raw.items():
    index_vecteurs[doc] = torch.tensor(vec, dtype=torch.float32)

print(f"Loaded {len(index_vecteurs)} document vectors.")

# ===== Recherche =====
def search(query, top_k=20):
    # Encoder la requête en torch.float32
    q_vec = model.encode(query, convert_to_tensor=True).float()

    results = []
    for doc, vec_t in index_vecteurs.items():
        # Similarité cosinus
        score = util.cos_sim(q_vec, vec_t).item()
        if score >= 0.0:

            results.append((doc, score))

    # Trier par score décroissant
    results.sort(key=lambda x: x[1], reverse=True)
    return results[:top_k]

# ===== Exemple d'utilisation =====
if __name__ == "__main__":
    query = input("Enter your query: ")
    results = search(query)

    print("\nTop results:")
    for doc, score in results:
        print(f"{doc} — {score:.4f}")


Loaded 3084 document vectors.
Enter your query: love 

Top results:
2575-romeo-and-juliet-william-shakespeare.pdf_page_117 — 0.7023
16 King Lear author William Shakespeare.pdf_page_180 — 0.6614
16 King Lear author William Shakespeare.pdf_page_65 — 0.6488
19 Pericles, Prince of Tyre author William Shakespeare.pdf_page_24 — 0.6205
21 The Two Noble Kinsmen author William Shakespeare.pdf_page_162 — 0.5683
10 The Tempest author William Shakespeare.pdf_page_104 — 0.5652
22 Henry VI, Part 1 author William Shakespeare.pdf_page_122 — 0.5602
23 Henry VI, Part 2 author William Shakespeare.pdf_page_130 — 0.5590
16 King Lear author William Shakespeare.pdf_page_131 — 0.5379
23 Henry VI, Part 2 author William Shakespeare.pdf_page_66 — 0.5363
22 Henry VI, Part 1 author William Shakespeare.pdf_page_65 — 0.5341
24 Henry VI, Part 3 author William Shakespeare.pdf_page_72 — 0.5326
10 The Tempest author William Shakespeare.pdf_page_68 — 0.5307
21 The Two Noble Kinsmen author William Shakespeare.pdf_page_70 

In [ ]:
def book_title(filename):
    try:
        clean_name = filename.replace(".pdf", "")
        if " author " in clean_name:
            title_part = clean_name.split(" author ")[0]
        else:
            title_part = clean_name

        words = title_part.split()

        if words and words[0].isdigit():
            words = words[1:]  # On garde tout sauf le 1er élément
        return " ".join(words)

    except Exception as e:
        return None

In [ ]:
import os
import json
import requests
DOSSIER_PDF = "/content/drive/MyDrive/ressources"
DOSSIER_META = "/content/drive/MyDrive/MesLivresMetadata"
os.makedirs(DOSSIER_META, exist_ok=True)
def chercher_metadonnees(titre):
    if not titre:
        return None

    url = "https://www.googleapis.com/books/v1/volumes"
    params = {"q": titre, "maxResults": 1} # On ne veut que le 1er résultat

    try:
        response = requests.get(url, params=params)
        data = response.json()

        # CORRECTION DU BUG "KeyError: items"
        if "items" not in data:
            print(f"   [API] Rien trouvé pour : {titre}")
            return None

        return data["items"][0]["volumeInfo"]

    except Exception as e:
        print(f"   [Erreur API] {e}")
        return None

# --- COEUR DU PROGRAMME ---
print("--- Début du traitement ---")

# Lister tous les fichiers du dossier
fichiers = [f for f in os.listdir(DOSSIER_PDF) if f.endswith(".pdf")]

for fichier in fichiers:
    print(f"\nTraitement de : {fichier}")

    # 1. Extraire le titre propre
    titre_propre = book_title(fichier)
    print(f"   Titre détecté : {titre_propre}")

    # 2. Chercher les infos
    meta = chercher_metadonnees(titre_propre)

    if meta:
        # 3. Sauvegarder en JSON
        # On garde le même nom que le PDF mais en .json
        nom_json = fichier.replace(".pdf", ".json")
        chemin_json = os.path.join(DOSSIER_META, nom_json)

        with open(chemin_json, 'w', encoding='utf-8') as f:
            json.dump(meta, f, ensure_ascii=False, indent=4)

        print(f"   ✅ Sauvegardé dans : {nom_json}")
    else:
        print("   ❌ Pas de métadonnées trouvées.")

print("\n--- Terminé ! ---")

--- Début du traitement ---

Traitement de : 1 The Comedy of Errors author William Shakespeare.pdf
   Titre détecté : The Comedy of Errors
   ✅ Sauvegardé dans : 1 The Comedy of Errors author William Shakespeare.json

Traitement de : 2 Titus Andronicus author William Shakespeare.pdf
   Titre détecté : Titus Andronicus
   ✅ Sauvegardé dans : 2 Titus Andronicus author William Shakespeare.json

Traitement de : 3 The Rape of Lucrece author William Shakespeare.pdf
   Titre détecté : The Rape of Lucrece
   ✅ Sauvegardé dans : 3 The Rape of Lucrece author William Shakespeare.json

Traitement de : 4 The Winter's Tale author William Shakespeare.pdf
   Titre détecté : The Winter's Tale
   ✅ Sauvegardé dans : 4 The Winter's Tale author William Shakespeare.json

Traitement de : 5 King John author William Shakespeare.pdf
   Titre détecté : King John
   ✅ Sauvegardé dans : 5 King John author William Shakespeare.json

Traitement de : 6 A Midsummer Night's Dream author William Shakespeare.pdf
   Titre

In [ ]:
import os, json, re
import fitz
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer
import numpy as np

#MODEL_NAME = "sentence-transformers/bert-base-nli-mean-tokens"
MODEL_NAME = "sentence-transformers/all-MiniLM-L6-v2"
model = SentenceTransformer(MODEL_NAME)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)

dossier = "/content/drive/MyDrive/ressources"
out_path = "/content/drive/MyDrive/index_books_chunks.json"

# paramètres de chunking (adaptables)
MAX_TOKENS = min(tokenizer.model_max_length, 512)  # sécurité
CHUNK_SIZE = 450
OVERLAP = 50

def nettoyer_texte(t):
    t = t.replace("\n", " ")
    t = re.sub(r'\s+', ' ', t)
    return t.strip()

def chunk_text_by_tokens(text, chunk_size=CHUNK_SIZE, overlap=OVERLAP):
    toks = tokenizer(text, add_special_tokens=False)["input_ids"]
    chunks = []
    start = 0
    L = len(toks)
    while start < L:
        end = min(start + chunk_size, L)
        chunk_ids = toks[start:end]
        chunk_text = tokenizer.decode(chunk_ids, skip_special_tokens=True, clean_up_tokenization_spaces=True)
        chunks.append((start, end, chunk_text))
        if end == L:
            break
        start = end - overlap
    return chunks

index = {}

pdfs = [f for f in os.listdir(dossier) if f.lower().endswith(".pdf")]
for pdf_file in pdfs:
    path = os.path.join(dossier, pdf_file)
    print("Processing", pdf_file)
    try:
        pdf = fitz.open(path)
    except Exception as e:
        print("Error opening", pdf_file, e)
        continue

    # concat toutes les pages (ou tu peux chunker page par page si tu préfères)
    full_text = ""
    page_offsets = []  # pour retrouver de quelle page vient chaque chunk (facultatif)
    for i, page in enumerate(pdf, start=1):
        txt = page.get_text()
        txt = nettoyer_texte(txt)
        if txt:
            # stocke offset de caractère approximatif
            page_offsets.append((len(full_text), i))
            full_text += " " + txt

    pdf.close()

    if not full_text.strip():
        continue

    # découper en chunks par tokens
    chunks = chunk_text_by_tokens(full_text, CHUNK_SIZE, OVERLAP)
    chunk_texts = [c[2] for c in chunks]

    # encoder par batch pour vitesse
    embeddings = model.encode(chunk_texts, convert_to_numpy=True, show_progress_bar=True)

    # agrégation: moyenne de tous les chunks (ou moyenne top-k)
    agg_embedding = embeddings.mean(axis=0)

    # construire structure à sauvegarder
    index[pdf_file] = {
        "agg_embedding": agg_embedding.tolist(),
        "chunks": [
            {
                "start_token": int(start),
                "end_token": int(end),
                "text_preview": (chunk_texts[idx][:300] + "...") if len(chunk_texts[idx])>300 else chunk_texts[idx],
                "page_guess": None  # optionnel: on peut approximer la page en fonction des offsets
            }
            for idx, (start, end, _ ) in enumerate(chunks)
        ],
        # on sauvegarde embeddings des chunks séparément (attention taille du fichier)
        "chunk_embeddings": [emb.tolist() for emb in embeddings]
    }

# sauvegarde
with open(out_path, "w", encoding="utf-8") as f:
    json.dump(index, f, ensure_ascii=False, indent=2)

print("Saved index to", out_path)


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

Processing 24 Henry VI, Part 3 author William Shakespeare.pdf


Batches:   0%|          | 0/3 [00:00<?, ?it/s]

Processing 25 romeo and juliet author william shakespeare.pdf


Batches:   0%|          | 0/3 [00:00<?, ?it/s]

Saved index to /content/drive/MyDrive/index_books_chunks.json
